### Implement DeepDream algorithm in Keras


In [0]:
from keras.applications import inception_v3
from keras import backend as K
import numpy as np
import scipy
from keras.preprocessing import image

#wont be training the model, so disable all training operations
K.set_learning_phase(0)

#Load the inception v3 network without the convolutional base, weights = pretrained ImageNet weights
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
87916544/87910968 [==============================] - 3s 0us/step


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Compute the loss; we want to maximize the loss during gradient ascent proces. Specifically, we maximize a weighted sum of the L2 norm of the activations of a set
of high-level layers. Lower layers result in geometric patterns, whereas higher layers
result in visuals observed in training data of the pretrained network

In [0]:
#setting up DeepDream configuration, how much the layer’s activation contributes to the loss to maximize.
#Layer names hardcoded in inceptionV3, obtained by model.summary()
layer_contributions = {
    'mixed2':0.2, 'mixed3':3.,
    'mixed4':2., 'mixed5':1.5,
}

#define the loss 
layer_dict = dict([(layer.name,layer) for layer in model.layers]) #create a dict to map layer names to layer instances

loss = K.variable(0.)
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  activation = layer_dict[layer_name].output  #Retrieve the layer's output
  
  scaling = K.prod(K.cast(K.shape(activation),'float32'))
  #Add the L2 norm of the features of a layer to the loss. Avoid border artifacts by only involving nonborder pixels in the loss
  loss += coeff * K.sum(K.square(activation[:,2: -2, 2: -2, :])) /scaling 

Setup Gradient ascent process

In [0]:
#tensor to hold the generated image
dream = model.input

#compute the gradients of dream wrt loss
grads = K.gradients(loss,dream)[0] 

#Normalize the gradients
grads /= K.maximum(K.mean(K.abs(grads)),1e-7)

#Setup keras function to retrieve the value of loss and gradients for an input image
outputs = [loss,grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values = outs[1] 
  return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
  """This function runs
gradient ascent for a
number of iterations"""
  for i in range(iterations):
    loss_value, grad_values = eval_loss_and_grads(x)
    if max_loss is not None and loss_value > max_loss:
      break
    print("...Loss value at",i,':', loss_value)
    x += step*grad_values
    
  return x


### Implement the DeepDream algorithm

Run gradient ascent over different successive scales

In [0]:
step = 0.01     # Gradient ascent step size
num_octave = 3  # No of scales at which to run gradient ascent
octave_scale = 1.4 # Size ratio between scales
iterations = 20  # No of ascent steps to run at each scale
max_loss = 10.  #If the loss grows larger than 10, interrupt the process to avoid ugly artifacts
base_image_path = '/content/gdrive/My Drive/Colab Notebooks/data/cc_elephant.jpg'
img = preprocess_image(base_image_path) # Load the base image into a Numpy array

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1,num_octave):
  shape = tuple([int(dim/(octave_scale**i)) for dim in original_shape])
  successive_shapes.append(shape)
  
successive_shapes = successive_shapes[::-1]    #reverse list of shapes so they're in increasing order

#Resizes the Numpy array of the image to the smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
  print ('Processing image shape', shape)
  img = resize_img(img,shape)  #scale up the dream image
  img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)  #run gradient ascent altering the dream
  
  upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape) #scales up the smaller version of the original image: it will be pixellated
  same_size_original = resize_img(original_img, shape)    # Computes the high-quality version of the original image at this size
  lost_detail = same_size_original - upscaled_shrunk_original_img # The difference between the two is the detail that was lost when scaling up
  
  img += lost_detail  #Reinjects lost detail into the dream
  shrunk_original_img = resize_img(original_img, shape)
  save_img(img, fname='/content/gdrive/My Drive/Colab Notebooks/data/dream_at_scale_' + str(shape)+'.png')

save_img(img, fname='/content/gdrive/My Drive/Colab Notebooks/data/final_dream.png')

Processing image shape (306, 458)
...Loss value at 0 : 1.8070962
...Loss value at 1 : 2.3078375
...Loss value at 2 : 3.0242286
...Loss value at 3 : 3.7770443
...Loss value at 4 : 4.5572577
...Loss value at 5 : 5.2607713
...Loss value at 6 : 5.9627633
...Loss value at 7 : 6.5625815
...Loss value at 8 : 7.200746
...Loss value at 9 : 7.76727
...Loss value at 10 : 8.364681
...Loss value at 11 : 8.898355
...Loss value at 12 : 9.465099
...Loss value at 13 : 9.956362


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


Processing image shape (428, 642)
...Loss value at 0 : 3.0523643
...Loss value at 1 : 4.382828
...Loss value at 2 : 5.5210733
...Loss value at 3 : 6.488146
...Loss value at 4 : 7.407243
...Loss value at 5 : 8.214131
...Loss value at 6 : 8.998534
...Loss value at 7 : 9.683636
Processing image shape (600, 899)
...Loss value at 0 : 2.9750822
...Loss value at 1 : 4.332281
...Loss value at 2 : 5.4620795
...Loss value at 3 : 6.464133
...Loss value at 4 : 7.4006147
...Loss value at 5 : 8.269527
...Loss value at 6 : 9.078883
...Loss value at 7 : 9.866173


Auxiliary functions

In [0]:
def resize_img(img, size):
  img = np.copy(img)
  factors = (1, float(size[0])/img.shape[1], float(size[1])/img.shape[2],1)
  return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
  pil_img = deprocess_image(np.copy(img))
  scipy.misc.imsave(fname, pil_img)
  
def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = inception_v3.preprocess_input(img)
  return img

def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1,2,0))
  else:
    x = x.reshape((x.shape[1],x.shape[2],3))
    
  x /= 2.
  x += 0.5
  x *= 255.
  x = np.clip(x,0,255).astype('uint8')
  return x

Next steps -

- Explore how output changes by changing which layers we use in the loss. 
